In [1]:
# Définir les paramètres de la courbe de Montgomery (Curve25519)
p = 2^255 - 19
A = 486662
F = GF(p)

# Définir un point de base P avec seulement la coordonnée x
xP = F(9)

def xADD(XP, ZP, XQ, ZQ, X_diff, p):
    F = GF(p)
    A = F(XP + ZP)
    B = F(XP - ZP)
    C = F(XQ + ZQ)
    D = F(XQ - ZQ)
    DA = D * A
    CB = C * B
    X_out = (DA + CB)^2
    Z_out = X_diff * (DA - CB)^2
    return X_out, Z_out



def xDBL(XP, ZP, a24, p):
    F = GF(p)
    A = F(XP + ZP)
    B = F(XP - ZP)
    AA = A^2
    BB = B^2
    E = AA - BB
    X_out = AA * BB
    Z_out = E * (BB + a24 * E)
    return X_out, Z_out


def montgomery_ladder(xP, A, n, p):
    F = GF(p)
    a24 = F((A + 2) / 4)

    # Représentation projective : (X : Z)
    X0, Z0 = F(1), F(0)      # Point à l'infini (0*P)
    X1, Z1 = F(xP), F(1)     # Point P

    n_bin = bin(n)[2:]       # Bits de n sans '0b'

    for bit in n_bin:
        bit = int(bit)
        if bit == 0:
            # P = P, Q = Q
            X1, Z1 = xADD(X0, Z0, X1, Z1, xP, p)
            X0, Z0 = xDBL(X0, Z0, a24, p)
        else:
            # P = Q, Q = P
            X0, Z0 = xADD(X1, Z1, X0, Z0, xP, p)
            X1, Z1 = xDBL(X1, Z1, a24, p)

    return X0 / Z0
